In [4]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")
chat = ChatOpenAI(model_name="gpt-3.5-turbo")

a = llm.predict("How many planets are there in the solar system?")
b = chat.predict("How many planets are there in the solar system?")

a, b

('\n\nThere are 8 planets in the solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are 8 planets in the solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

In [5]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
	SystemMessage(content="You are a professional italian chef."),
	AIMessage(content="Welcome to the italian restaurant. What would you like to order?"),
	HumanMessage(content="I'd like to order a pizza. What are the toppings?"),
]

chat.predict_messages(messages)

AIMessage(content='We have a variety of toppings available for our pizzas. Some popular options include classic Margherita with tomato sauce, fresh mozzarella, and basil, or a Quattro Stagioni with artichokes, mushrooms, ham, and olives. We also have options like Prosciutto e Rucola with prosciutto, arugula, and shaved Parmesan, or a Diavola with spicy salami and chili flakes. Feel free to customize your pizza with any toppings you prefer!')

In [6]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("What is the distance between the {planet_a} and the {planet_b}?",)

prompt = template.format(planet_a="Earth", planet_b="Mars")

chat.predict(prompt)

'The distance between Earth and Mars varies depending on their positions in their respective orbits. On average, the distance between Earth and Mars is about 225 million kilometers (140 million miles). However, this distance can range from about 54.6 million kilometers (33.9 million miles) when the two planets are at their closest approach (opposition) to about 401 million kilometers (249 million miles) when they are at their farthest apart (conjunction).'

In [7]:
template = ChatPromptTemplate.from_messages([
	("system", "You are a professional {nationality} chef."),
	("ai", "Welcome to the {nationality} restaurant. What would you like to order?"),
	("human", "I'd like to order a {food}. What are the ingredients?"),
])

prompt = template.format_messages(nationality="italian", food="pizza")

chat.predict_messages(prompt)

AIMessage(content='For a classic Italian pizza, we typically use a thin crust topped with tomato sauce, mozzarella cheese, and various toppings such as pepperoni, mushrooms, bell peppers, onions, olives, and basil. Is there a specific type of pizza you have in mind or any dietary restrictions I should be aware of?')

In [8]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
	def parse(self, text: str) -> str:
		items = text.strip().split(",")
		return list(map(str.strip, items))

p = CommaOutputParser()

p.parse("Hello,how,are,you?")

['Hello', 'how', 'are', 'you?']

In [12]:
template = ChatPromptTemplate.from_messages([
	("system", """You are a list generating machine. 
	Everything you are asked will be answered with a comma separated list of max {max_items}, in lowercase. Do not reply with anything else.
	"""),
	("human", "{question}")
])

prompt = template.format_messages(
	max_items=10,
	question="What are the colors?"
)

output = chat.predict_messages(prompt)

p.parse(output.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'black',
 'white',
 'pink',
 'brown']

In [14]:
chain = template | chat | CommaOutputParser()

chain.invoke({
	"max_items": 10,
	"question": "What are the colors?"
})


['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [16]:
chef_prompt = ChatPromptTemplate.from_messages([
	("system", "You are a professional international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
	("human", "I want to cook a {cuisine} dish. What is the best recipe for beginner?"),
])

chef_chain = chef_prompt | chat

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
	("system", """
		You are a vegetarian chef specialized on making traditional recipes vegetarian. 
		You find alternative ingredients and explain their preparation. 
		You don't radically modify the recipe. 
		If there is no alternative for a food just say you don't know how to replace it
	"""),
	("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chain

final_chain.invoke({
	"cuisine": "korean",
})
